In [50]:
import pandas 
import numpy
import re
import unicodedata
import string
import spacy
import nltk
from nltk.tokenize import ToktokTokenizer
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('not')

nlp = spacy.load('en_core_web_sm')
data = pandas.read_csv("Original Data\Book20_western.csv")

In [52]:
X = pandas.DataFrame(data, columns=data.columns)
print(X)

                                                reviews ratings  \
0     Great direction, screenplay, performances, sou...       8   
1     For anyone who isn't much into cinema, I would...      10   
2     (Jamie Fox, Christoph Waltz, Leonardo DiCaprio...      10   
3     I just saw this film and I confess that I am c...       8   
4     Quentin Tarantino, one of the most iconic dire...       0   
...                                                 ...     ...   
6709  Much will be said about this film. Whether it ...       0   
6710  You'd think this was a filming of "The Emperor...       1   
6711  The Revenant is beautifully shot, but the stor...       3   
6712  I had high hopes for this film given fuss abou...       3   
6713  First of all, I very much like survival movies...       4   

                                               subjects            titles  
0      Great film with the crazy passion of a great ...  Django Unchained  
1                                   The Per

In [53]:
X.shape

(6714, 4)

In [54]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

def fix_int(every_ratings):
    every_ratings = re.sub("[^A-Za-z0-9]","",every_ratings)
    every_ratings = re.sub("[^0-9]","0",every_ratings)[:2]
    every_ratings = int(every_ratings)
    return every_ratings

def remove_special_characters(text):
    # define the pattern to keep
    pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]' 
    return re.sub(pat,'', text)

def remove_punctuation(text):
    text = ''.join([c for c in text if c not in string.punctuation])
    return text

def remove_numbers(text):
    # define the pattern to keep
    pattern = r'[^a-zA-z.,!?/:;\"\'\s]' 
    return re.sub(pattern, '', text)    

def get_lem(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

def remove_stopwords(text):
    # convert sentence into token of words
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    # check in lowercase 
    t = [token for token in tokens if token.lower() not in stopword_list]
    text = ' '.join(t)    
    return text

In [55]:
words4 = "\""
words5 = "\b"
words6 = "\'"
for i in data.index:
    data.loc[i,'ratings'] = re.sub("[^A-Za-z0-9]","",data.loc[i,'ratings'])
    data.loc[i,'ratings'] = re.sub("[^0-9]","0",data.loc[i,'ratings'])[:2]
    data.loc[i,'ratings'] = fix_int(data.loc[i,'ratings'])
    data.loc[i,'reviews'] = unicodedata.normalize('NFKD',data.loc[i,'reviews']).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    data.loc[i,'reviews'] = decontracted(data.loc[i,'reviews'])
    data.loc[i,'reviews'] = remove_special_characters(data.loc[i,'reviews'])
    data.loc[i,'reviews'] = remove_punctuation(data.loc[i,'reviews'])
    data.loc[i,'reviews'] = remove_numbers(data.loc[i,'reviews'])
    data.loc[i,'reviews'] = get_lem(data.loc[i,'reviews'])
    data.loc[i,'reviews'] = remove_stopwords(data.loc[i,'reviews'])
    data.loc[i,'subjects'] = unicodedata.normalize('NFKD',data.loc[i,'subjects']).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    data.loc[i,'subjects'] = decontracted(data.loc[i,'subjects'])
    data.loc[i,'subjects'] = remove_special_characters(data.loc[i,'subjects'])
    data.loc[i,'subjects'] = remove_punctuation(data.loc[i,'subjects'])
    data.loc[i,'subjects'] = remove_numbers(data.loc[i,'subjects'])
    data.loc[i,'subjects'] = get_lem(data.loc[i,'subjects'])
    data.loc[i,'subjects'] = remove_stopwords(data.loc[i,'subjects'])
    data.loc[i,'titles'] = unicodedata.normalize('NFKD',data.loc[i,'titles']).encode('ascii', 'ignore').decode('utf-8', 'ignore')

In [56]:
data.reviews[500]

'funny Jamie Foxx explain Franco Nero silent tip hat original Django plot completely different though sort origin story maybe make Django liberated black slave try free wife help german bounty hunterhere Tarantino team Christoph Waltz guy good role far Inglorious bastard familiar Tarantino expect lot flamboyant dialogue follow gun shot could one good film lately mainly not tarantine almost feel like normal movie not talking seemingly inadvertent humourBottom line western doubt Tarantino unmistakable style add fun black guy hero stellar cast help well Jamie Foxx Leo di Caprio Samuel L Jackson role could possibly carry well course Tarantino cameo small role possible kudo recognize James Remar role always expect connection two character not'

In [57]:
data = list(zip(data.reviews, data.subjects, data.ratings, data.titles))

dataframe = pandas.DataFrame(data, columns=["reviews","subjects","ratings","titles"])

dataframe.to_csv(r'C:\Users\User\Downloads\Review\Updated Data\updated_reviews_western.csv',index=False, header=True)